# SQL Parser using Lark
### Lark
- Ref: [Lark Documentation](https://lark-parser.readthedocs.io/en/latest/)
  - Repo: [lark-parser](https://github.com/lark-parser/lark)

### Lark SQL Parser
- Ref: [sql_to_ibis](https://github.com/zbrookle/sql_to_ibis)
- Ref: [ibis](https://github.com/ibis-project/ibis)
  - Docs: https://ibis-project.org/docs/dev/

`sql_to_ibis` is a [Python](https://www.python.org/) package that translates SQL syntax into [ibis](https://github.com/ibis-project/ibis) expressions. This provides the capability of using only one SQL dialect to target many different backends.

- Ref: From `sql_to_ibis` project `sql_select_query.py`
- Ref: SQL grammar file <https://github.com/zbrookle/sql_to_ibis/tree/main/sql_to_ibis/grammar>

## Firebase Client

In [7]:
from firesql.firebase import FirebaseClient
from firesql.sql.sql_fire_client import FireSQLClient

firebaseClient = FirebaseClient()
firebaseClient.connect(credentials_json='../credentials/credentials.json')

# create FireSQLClient that is using the FireSQLAbstractClient interface.
client = FireSQLClient(firebaseClient)

In [1]:
import os
from lark import Lark, tree

_ROOT = "../firesql/sql"
GRAMMAR_PATH = os.path.join(_ROOT, "grammar", "firesql.lark")
with open(file=GRAMMAR_PATH) as sql_grammar_file:
    _GRAMMAR_TEXT = sql_grammar_file.read()
parser = Lark(_GRAMMAR_TEXT, parser="lalr")

## SQL Insert

In [2]:
from firesql.sql.sql_transformer import SQLTransformer

company = 'bennycorp'

sql1 = """
  INSERT INTO Companies/bennycorp/Tests (email, state, event)
    VALUES ( 'btscheung+test1@gmail.com', 'ACTIVE', JSON(["a","b","c"]) )
"""

sql2 = """
  INSERT INTO Companies/bennycorp/Tests (*)
    VALUES (
      JSON(
        {
          "access": {
              "hasAccess": true
          },
          "assignedSeats": [],
          "email": "btscheung+twotwo@gmail.com",
          "emailVerified": true,
          "externalId": null,
          "firstName": "Benny",
          "groups": [],
          "lastName": "TwoTwo",
          "preferredSeats": [],
          "roles": [
              "CLEANER"
          ],
          "state": "ACTIVE",
          "vaccination": null
      }
      )
    )
"""

ast = parser.parse(sql2)
statements = SQLTransformer().transform(ast)
statements[0]

SQL_Insert(table=SQL_SelectFrom(part='Companies/bennycorp/Tests', alias=None), columns=[SQL_ColumnRef(table=None, column='*', func=None)], values=[SQL_ValueJSON(value={'access': {'hasAccess': True}, 'assignedSeats': [], 'email': 'btscheung+twotwo@gmail.com', 'emailVerified': True, 'externalId': None, 'firstName': 'Benny', 'groups': [], 'lastName': 'TwoTwo', 'preferredSeats': [], 'roles': ['CLEANER'], 'state': 'ACTIVE', 'vaccination': None})])

In [3]:
from firesql.sql import SQLFireInsert

sqlCommand = SQLFireInsert()
sqlCommand.generate(statements[0], options={})

True

In [4]:
print(sqlCommand.part)
print(sqlCommand.columns)
print(sqlCommand.values)

Companies/bennycorp/Tests
['*']
[{'access': {'hasAccess': True}, 'assignedSeats': [], 'email': 'btscheung+twotwo@gmail.com', 'emailVerified': True, 'externalId': None, 'firstName': 'Benny', 'groups': [], 'lastName': 'TwoTwo', 'preferredSeats': [], 'roles': ['CLEANER'], 'state': 'ACTIVE', 'vaccination': None}]


In [5]:
document = sqlCommand.post_process()
print(document)

{'access': {'hasAccess': True}, 'assignedSeats': [], 'email': 'btscheung+twotwo@gmail.com', 'emailVerified': True, 'externalId': None, 'firstName': 'Benny', 'groups': [], 'lastName': 'TwoTwo', 'preferredSeats': [], 'roles': ['CLEANER'], 'state': 'ACTIVE', 'vaccination': None}


In [13]:
insertedDoc = sqlCommand.execute(client, document)
insertedDoc

{'access': {'hasAccess': True},
 'assignedSeats': [],
 'email': 'btscheung+twotwo@gmail.com',
 'emailVerified': True,
 'externalId': None,
 'firstName': 'Benny',
 'groups': [],
 'lastName': 'TwoTwo',
 'preferredSeats': [],
 'roles': ['CLEANER'],
 'state': 'ACTIVE',
 'vaccination': None,
 'docid': 'Mskj0W8fH925626UQSVG'}

## SQL Update

In [9]:
from firesql.sql.sql_transformer import SQLTransformer

company = 'bennycorp'

sql1 = """
  UPDATE Companies/bennycorp/Tests
  SET state = 'INACTIVE'
  WHERE state = 'ACTIVE' and email = 'btscheung+test1@gmail.com'
"""

ast = parser.parse(sql1)
statements = SQLTransformer().transform(ast)
statements[0]

SQL_Update(table=SQL_SelectFrom(part='Companies/bennycorp/Users', alias=None), sets=[SQL_BinaryExpression(operator='==', left=SQL_ColumnRef(table=None, column='state', func=None), right=SQL_ValueString(value='INACTIVE'))], where=SQL_BinaryExpression(operator='and', left=SQL_BinaryExpression(operator='==', left=SQL_ColumnRef(table=None, column='state', func=None), right=SQL_ValueString(value='ACTIVE')), right=SQL_BinaryExpression(operator='==', left=SQL_ColumnRef(table=None, column='email', func=None), right=SQL_ValueString(value='btscheung+test1@gmail.com'))))

In [10]:
from firesql.sql import SQLFireUpdate  

sqlCommand = SQLFireUpdate()
queries = sqlCommand.generate(statements[0], options={})

In [11]:
fireQueries = sqlCommand.firebase_queries(queries)
fireQueries

{'Companies/bennycorp/Users': [['state', '==', 'ACTIVE'],
  ['email', '==', 'btscheung+test1@gmail.com']]}

In [12]:
filterQueries = sqlCommand.filter_queries(queries)
filterQueries

{'Companies/bennycorp/Users': []}

In [13]:
documents = sqlCommand.execute_query(client, fireQueries)

In [14]:
filterDocs = sqlCommand.filter_documents(documents, filterQueries)
filterDocs

{'Companies/bennycorp/Users': {'KZkujJaukQBQdGtf559U': {'events': [{'event': 'CREATION',
     'date': DatetimeWithNanoseconds(2022, 4, 23, 1, 35, 8, 627000, tzinfo=datetime.timezone.utc)}],
   'email': 'btscheung+test1@gmail.com',
   'state': 'ACTIVE'}}}

In [16]:
selectDocs = sqlCommand.post_process(filterDocs)

In [17]:
sqlCommand.select_fields()

['docid', 'email', 'events', 'state']

In [18]:
from firesql.sql import DocPrinter

docPrinter = DocPrinter()
docPrinter.printCSV(selectDocs, sqlCommand.select_fields())

"docid","email","events","state"
"KZkujJaukQBQdGtf559U","btscheung+test1@gmail.com","{\"event\": \"CREATION\", \"date\": \"2022-04-23T01:35:08\"}","INACTIVE"


In [19]:
updatedDocs = sqlCommand.execute(client, filterDocs)

## SQL Delete

In [20]:
from firesql.sql.sql_transformer import SQLTransformer

company = 'bennycorp'

sql1 = """
  DELETE FROM Companies/{}/Tests
    WHERE 
      email = 'btscheung+test1@gmail.com'
""".format(company)

ast = parser.parse(sql1)
statements = SQLTransformer().transform(ast)
statements[0]

SQL_Delete(table=SQL_SelectFrom(part='Companies/bennycorp/Users', alias=None), where=SQL_BinaryExpression(operator='==', left=SQL_ColumnRef(table=None, column='email', func=None), right=SQL_ValueString(value='btscheung+test1@gmail.com')))

In [21]:
from firesql.sql import SQLFireDelete

sqlCommand = SQLFireDelete()
queries = sqlCommand.generate(statements[0], options={})


In [22]:
print(sqlCommand.collections)
print(queries)

{'Companies/bennycorp/Users': 'Companies/bennycorp/Users'}
{'Companies/bennycorp/Users': [['email', '==', 'btscheung+test1@gmail.com']]}


In [23]:
fireQueries = sqlCommand.firebase_queries(queries)
fireQueries

{'Companies/bennycorp/Users': [['email', '==', 'btscheung+test1@gmail.com']]}

In [24]:
filterQueries = sqlCommand.filter_queries(queries)
filterQueries

{'Companies/bennycorp/Users': []}

In [25]:
documents = sqlCommand.execute_query(client, fireQueries)

In [26]:
filterDocs = sqlCommand.filter_documents(documents, filterQueries)
filterDocs

{'Companies/bennycorp/Users': {'KZkujJaukQBQdGtf559U': {'events': [{'event': 'CREATION',
     'date': DatetimeWithNanoseconds(2022, 4, 23, 1, 35, 8, 627000, tzinfo=datetime.timezone.utc)}],
   'state': 'INACTIVE',
   'email': 'btscheung+test1@gmail.com'}}}

In [27]:
selectDocs = sqlCommand.post_process(filterDocs)

In [28]:
sqlCommand.select_fields()

['docid', 'email', 'events', 'state']

In [29]:
from firesql.sql import DocPrinter

docPrinter = DocPrinter()
docPrinter.printCSV(selectDocs, sqlCommand.select_fields())

"docid","email","events","state"
"KZkujJaukQBQdGtf559U","btscheung+test1@gmail.com","{\"event\": \"CREATION\", \"date\": \"2022-04-23T01:35:08\"}","INACTIVE"


In [30]:
deletedDocs = sqlCommand.execute(client, filterDocs)